In [ ]:
# Load AIEEE Data

In [1]:
%cd ../../

/home/souvic/mounted/btp/vahini/Name2Demographics


In [18]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Fri Apr  1 10:45:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/

In [19]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"    
import tensorflow as tf

In [3]:
import sys  
sys.path.insert(0, 'Models/ERData/PreProcessing/')

from er_preprocess import *

In [4]:
er_df = pd.read_csv("Models/Data/ERData/er_data_with_majority_label.csv")

Load Models

In [5]:
#  other models
AIEEE_Muril = "Models/AIEEEGenderData/MurilBERT/SavedModel/MurilBERT"
CBSE_Muril = "Models/CBSEData/MurilBERT/SavedModel/MurilBERT"
ER_Muril = "Models/ERData/MurilBERT/SavedModel/BERTMuril"

In [6]:
%%capture
!pip install transformers[sentencepiece]
import transformers

In [7]:
from transformers import BertModel, AutoModelForSequenceClassification
from transformers import AutoTokenizer

In [8]:
aieee_model = AutoModelForSequenceClassification.from_pretrained(AIEEE_Muril)

In [9]:
cbse_model = AutoModelForSequenceClassification.from_pretrained(CBSE_Muril)

In [10]:
er_model = AutoModelForSequenceClassification.from_pretrained(ER_Muril)

In [11]:
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")

In [12]:
er_df = er_df.dropna()

Predict Gender

In [13]:
from transformers import pipeline
from sklearn.metrics import classification_report


In [22]:
import math

def print_results(rmodel, tokenizer, df):
    pipe = pipeline(task="text-classification",model=rmodel, tokenizer=tokenizer, device=0)
    n = df.shape[0]
    predictions = []
    for i in range(math.ceil(n/1000)):
        df1 = df[i*1000:(i+1)*(1000)]
        onway = pipe(list(df1['Name']), batch_size=256)
        predictions = predictions + onway
        with open("Models/ERData/er_data_aieee_test1.txt", 'a') as f:
            f.write(f"{i}\n")
    labels = []
    for pred in predictions:
        if pred['label'] == 'LABEL_0':
            labels.append(0)
        else:
            labels.append(1)
    
    true = list(df['Gender'])
    report = classification_report(true, labels, digits = 4)
    print(classification_report(true, labels, digits = 4))
    with open("Models/ERData/er_data_aieee_test1.txt", 'a') as f:
            f.write(f"{report}\n")
    return report

In [ ]:
#  5318

In [25]:
report1 = print_results(aieee_model, tokenizer, er_df[5318000:])

              precision    recall  f1-score   support

         0.0     0.8384    0.8801    0.8587    841103
         1.0     0.8937    0.8559    0.8744    990385

    accuracy                         0.8670   1831488
   macro avg     0.8660    0.8680    0.8665   1831488
weighted avg     0.8683    0.8670    0.8672   1831488



In [16]:
report2 = print_results(aieee_model, tokenizer, er_df[:2000000])

              precision    recall  f1-score   support

         0.0     0.7812    0.7227    0.7508    999994
         1.0     0.7420    0.7975    0.7688   1000006

    accuracy                         0.7601   2000000
   macro avg     0.7616    0.7601    0.7598   2000000
weighted avg     0.7616    0.7601    0.7598   2000000



In [23]:
report3 = print_results(aieee_model, tokenizer, er_df[2000000:5318000])

              precision    recall  f1-score   support

         0.0     0.7543    0.7570    0.7557   1495365
         1.0     0.8001    0.7977    0.7989   1822635

    accuracy                         0.7794   3318000
   macro avg     0.7772    0.7774    0.7773   3318000
weighted avg     0.7794    0.7794    0.7794   3318000



In [18]:
print(report2)

              precision    recall  f1-score   support

         0.0     0.7812    0.7227    0.7508    999994
         1.0     0.7420    0.7975    0.7688   1000006

    accuracy                         0.7601   2000000
   macro avg     0.7616    0.7601    0.7598   2000000
weighted avg     0.7616    0.7601    0.7598   2000000



In [ ]:
print(report3)

In [ ]:
print(er_df.shape)

In [15]:
er_df.shape

(7149489, 3)

In [17]:
er_df.dropna().shape

(7149488, 3)

In [18]:
er_df = er_df.dropna()

In [19]:
er_df.shape

(7149488, 3)

In [ ]:
p